In [ ]:
import sys; sys.path.append('..')
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time, pickle
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm

In [ ]:
elastic_rods.set_max_num_tbb_threads(1)
linkage = pickle.load(open('../data/nonuniform_debug_convergence.pkl', 'rb'))
driver = 64
view = linkage_vis.LinkageViewer(linkage)
view.setCameraParams(((-0.7239722319967554, -2.953428306702387, -0.5451540221763805),
 (0.028326464489972513, -0.2362002372339648, -0.9712914389307234),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
linkage.joint(driver).alpha = 0.5574418589567819
jdo = linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint

In [ ]:
equilibriumSolver = lambda l, nit, verbose, fv: elastic_rods.compute_equilibrium(l, nit, verbose, useIdentityMetric=True, beta=1e-8, useNegativeCurvatureDirection=True, fixedVars=fv)

In [ ]:
cr = equilibriumSolver(linkage, 10, True, fixedVars)
view.update()

In [ ]:
from scipy.sparse.linalg import spsolve
Htrip = linkage.hessian()
Htrip.rowColRemoval(fixedVars)
Htrip.reflectUpperTriangle()
Hfree = csc_matrix(Htrip.compressedColumn())
freeGradient = lambda l, updatedSource: np.delete(l.gradient(updatedSource), fixedVars)
pfree = spsolve(Hfree, -freeGradient(linkage, True))
p = np.zeros(linkage.numDoF())
p[np.delete(np.arange(linkage.numDoF()), fixedVars)] = pfree

In [ ]:
def energyAt(l, dof, etype = elastic_rods.EnergyType.Full, updateSource = False):
    lcopy = elastic_rods.RodLinkage(l)
    lcopy.setDoFs(dof)
    return lcopy.energy(etype)
def gradientAt(l, dof, etype = elastic_rods.EnergyType.Full, updateSource = False):
    lcopy = elastic_rods.RodLinkage(l)
    lcopy.setDoFs(dof)
    if (updateSource): lcopy.updateSourceFrame()
    return freeGradient(lcopy, updateSource)
def fd_hessian_test(l, stepSize, direction, etype = elastic_rods.EnergyType.Full, updateSource = False):
    H = l.hessian()
    H.rowColRemoval(fixedVars)
    H.reflectUpperTriangle()
    H = csc_matrix(H.compressedColumn())
    dirfree = np.delete(direction, fixedVars)
    dof = l.getDoFs()
    #return [(gradientAt(l, dof + stepSize * direction, etype, updateSource) - gradientAt(l, dof - stepSize * direction, etype, updateSource)) / (2 * stepSize), H * dirfree]
    return [(gradientAt(l, dof + stepSize * direction, etype, updateSource) - gradientAt(l, dof, etype, updateSource)) / stepSize, H * dirfree]

In [ ]:
gradientAt(linkage, linkage.getDoFs() + 0.4 * p, elastic_rods.EnergyType.Full)

In [ ]:
fd_hessian_test(linkage, 0.25, p)

In [ ]:
gradNormAlongLine = lambda l, direction, stepSize, updateSource = False: norm(gradientAt(l, l.getDoFs() + stepSize * direction, updateSource=updateSource))
energyAlongLine   = lambda l, direction, stepSize: norm(energyAt(l, l.getDoFs() + stepSize * direction))

In [ ]:
steps = np.linspace(0, 1.1, 250)
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.title('Gradient norm evaluated on search ray')
plt.ylabel('Gradient norm')
plt.xlabel('Distance')
plt.plot(steps, [gradNormAlongLine(linkage, p, s, False) for s in steps])
plt.show()
#plt.plot(steps, [gradNormAlongLine(linkage, p, s, True) for s in steps])
#plt.show()
plt.title('Energy evaluated on search ray')
plt.ylabel('Energy')
plt.xlabel('Distance')
plt.plot(steps, [energyAlongLine(linkage, p, s) for s in steps])
plt.show()

In [ ]:
linkage = pickle.load(open('../data/nonuniform_debug_convergence.pkl', 'rb'))
linkage.joint(driver).alpha = 0.5574418589567819
cr = equilibriumSolver(linkage, 100, True, fixedVars)
view.update()

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
import convergence_reporting
convergence_reporting.plot_energy(cr)
plt.show()
convergence_reporting.plot_gradnorm(cr)
plt.show()
plt.ylabel('Structure height')
plt.xlabel('Newton iterations')
heights = [d['bb_size_2'] for d in cr.customData]
plt.scatter(np.arange(len(heights)), heights, marker='+', c=cr.indefinite, cmap='rainbow')
plt.show()